In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import pickle

In [2]:
df = pd.read_csv("KAG_conversion_data.csv")

In [3]:
df = df[df["xyz_campaign_id"]==1178]
df["CVR"] = df["Approved_Conversion"] / df["Clicks"]
df["CPC"] = df["Spent"] / df["Clicks"]
appConVal = df["Approved_Conversion"] * 50
df["ROAS"] = appConVal / df["Spent"]

In [4]:
df_filt = df[df["ROAS"] != float('inf')]
df_filt = df_filt.dropna(axis=0)

In [5]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
encoder.fit(df_filt["gender"])
df_filt["gender"]=encoder.transform(df_filt["gender"])
encoder.fit(df_filt["age"])
df_filt["age"]=encoder.transform(df_filt["age"])

In [6]:
x=np.array(df_filt.drop(labels=["ad_id", "xyz_campaign_id", "fb_campaign_id", "Impressions", "Clicks", "Spent", "Total_Conversion", "Approved_Conversion", "CVR"], axis=1))
y=np.array(df_filt["CVR"])

In [7]:
y=y.reshape(len(y),1)
from sklearn.impute import SimpleImputer
my_imp = SimpleImputer()
y_imp = my_imp.fit_transform(y)

In [8]:
from sklearn.preprocessing import StandardScaler
x_sc = StandardScaler()
x = x_sc.fit_transform(x)
x_imp = my_imp.fit_transform(x)

In [9]:
x_train, x_test, y_train, y_test = train_test_split(x_imp,y_imp,test_size=0.2, random_state=42)

In [10]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state=0)
rf.fit(x_train, y_train)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)

In [11]:
y_pred = rf.predict(x_test)

In [12]:
rf.score(x_test,y_test)

0.9579623415149814

In [13]:
import pickle
pickle.dump(rf, open('model.pkl', 'wb'))